In [1]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt  
import matplotlib.cbook as cbook
import numpy as np
import os
import sys
import requests
import json
import datawash

In [28]:
AKEY = 'Pt15VZ8Msk0fjKO4SXLUD58CiwbOOZ0P'
def get_coord(address):
    """从百度地图API获取地址对应的坐标。
    """
    response = requests.get('http://api.map.baidu.com/geocoder/v2/?address=' + address 
                            + '&ak=' + AKEY + '&output=json')
    address_dict = json.loads(response.text)
    return address_dict

def get_map():
    """从百度地图API获取地图并存储
    """
    width = '1024'
    height = '768'
    centerlng = 105.0
    centerlat = 37.0
    response = requests.get('http://api.map.baidu.com/staticimage/v2?ak=' + AKEY
                            + '&width='+ width + '&height='+ height
                            + '&zoom=5&center=' + str(centerlng)+','+str(centerlat))
    with open(os.path.join('image','map.png'),'wb') as img:
        img.write(response.content)
    edge = {'left':67.177, 'right':142.606,'top':56.269,'bottom':11.467}
    return os.path.abspath(os.path.join('image','map.png')),edge

In [29]:
# 遍历所有用户，读取需要的信息
locationslist = list()

jsons = datawash.datajsons()
for user in jsons:
    try:
        locationslist.append(user['locations'][0]['name'])
    except:
        pass

In [34]:
# 批量获取坐标
coord_dict = dict()
templist = list(set(locationslist))
for loc in templist:
    coord_dict[loc] = get_coord(loc)

ConnectionError: HTTPConnectionPool(host='api.map.baidu.com', port=80): Max retries exceeded with url: /geocoder/v2/?address=%E6%B0%AA%E6%98%9F%E7%90%83&ak=Pt15VZ8Msk0fjKO4SXLUD58CiwbOOZ0P&output=json (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001F77A058358>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。',))

In [36]:
# 获取地图
img = get_map()
imgfile = img[0]
imgedge = img[1]
# 绘制地图
image_file = cbook.get_sample_data(imgfile)  
image = plt.imread(image_file)
fig, ax= plt.subplots(1,1)
ax.imshow(image)
# 绘制用户分布
# 'lng':data['result']['location']['lng']
# 'lat':data['result']['location']['lat']
x = list()
y = list()

d = [11.467, 22.195, 32.429, 41.558, 49.36, 56.269]
for loc in locationslist:
    try:
        if coord_dict[loc]['status'] == 0:
            lng = coord_dict[loc]['result']['location']['lng']+np.random.rand(1)[0]*2-1
            lat = coord_dict[loc]['result']['location']['lat']+np.random.rand(1)[0]*2-1
            tempx =((lng-imgedge['left'])/(imgedge['right']-imgedge['left'])) * 1024
            for i in range(len(d)-1):
                if lat > d[i] and lat <= d[i+1]:
                    tempy = (((d[i+1]-lat)/(d[i+1]-d[i]))/5.0+(1-(i+1)/5.0))*768
            x.append(tempx)
            y.append(tempy)
    except:
        pass

x = np.array(x)
y = np.array(y)
ax.scatter(x, y, color='#FF8C00',alpha=0.1, edgecolors='none')
ax.axis('off')
fig.show()